# Random Forest Classifier

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# imports and path setup
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
import tqdm
from sklearn.utils import shuffle
from joblib import Parallel, delayed
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from Pylette import extract_colors

from userkits.features import *
from userkits.utils import *

In [3]:
HALF_SIZE = False

In [4]:
# load data from train and eval directories
# set half=True to resize images to half to reduce memory usage
X, y = load_train_data(data_dir="../train_data", half=HALF_SIZE)
X, y = shuffle(X, y, random_state=42)

Loading train data: 100%|██████████| 29/29 [00:26<00:00,  1.09it/s]


In [5]:
def extract_features(images):
    def process_image(img):
        feats = []
        
        feats.extend(color_histogram(img))
        feats.extend(lbp_texture_features(img))
        feats.extend(find_mean(img))
        feats.extend(find_stddev(img))
        feats.extend(hsv_histogram(img))
        

        feats.extend(biome_specific_color_ratios(img))  
        feats.append(brightness(img))
        feats.append(saturation_mean(img))
        feats.append(dominant_hue(img))
        

        feats.extend(color_palette(img, palette_size=5)) 
        feats.extend(hsv_palette(img, palette_size=5))    
        

        feats.append(shannon_entropy(img))
        feats.append(edge_density(img))
        feats.append(texture_complexity(img))
        feats.append(color_diversity_index(img))
        feats.extend(block_pattern_detection(img))        
        
        
        
        return np.array(feats)

    features_list = Parallel(n_jobs=-1)(delayed(process_image)(img) for img in tqdm.tqdm(images, desc="Extracting features"))
    return np.array(features_list)

In [6]:

X_features = extract_features(X)
X_features.shape

Extracting features:   0%|          | 0/1483 [00:00<?, ?it/s]

Extracting features: 100%|██████████| 1483/1483 [02:02<00:00, 12.06it/s]


(1483, 5318)

Exception ignored in: <function ResourceTracker.__del__ at 0x110065bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x105f39bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes
Exception ignored in: <function ResourceTracker.__del__ at 0x107665bc0>
Traceback (most recent call last

In [7]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_encoded, test_size=0.2) # you can change test_size
clf = RandomForestClassifier() # you can tune hyperparameters here
clf.fit(X_train, y_train)
print("Train Accuracy:", clf.score(X_train, y_train))
print("Test Accuracy:", clf.score(X_test, y_test))

Train Accuracy: 1.0
Test Accuracy: 0.9461279461279462


Exception ignored in: <function ResourceTracker.__del__ at 0x106e29bc0>
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 82, in __del__
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 91, in _stop
  File "/opt/anaconda3/lib/python3.13/multiprocessing/resource_tracker.py", line 116, in _stop_locked
ChildProcessError: [Errno 10] No child processes


## Evaluate

In [ ]:
# load eval data
# set half=True to resize images to half to reduce memory usage
X_eval, file_ids = load_eval_data("../eval_data", half=HALF_SIZE) 

In [ ]:
X_eval_features = extract_features(X_eval)
eval_predictions = clf.predict(X_eval_features)
print(eval_predictions[:5])

In [ ]:
try:
    preds = label_encoder.inverse_transform(eval_predictions)
except Exception:
    preds = eval_predictions

save_predictions(preds, file_ids, output_file='../output/rf_predictions.csv')